In [2]:
import sqlite3
import pandas as pd
import numpy as np
import docx

Для данной задачи выбран алгоритм:


1.   Написать чистый запрос на SQL (требование ТЗ)
2.   Создание Dataframe на Pandas (требование ТЗ). Для получения нужных сведений совершаются манипуляции средствами Pandas, Numpy и вспомогательной функции (см комметарии по тексту).

Как другие варианты, можно было попробовать часть задач реализовать внутри SQL чистыми запросами или некоторые манипуляции совершать на нативном Python или записи данных в отдельные файлы. Однако, Pandas для таких целей подходит лучше всего




In [110]:
connection = sqlite3.connect('test.db')
query = 'SELECT * FROM testidprod WHERE partner is NULL AND state is NULL AND bs = 0 AND (factor = 1 OR factor = 2)'
df = pd.read_sql_query(query, connection)
df

,id,country,factor,year,res,mir,raw,hash,meta,partner,state,bs
0,1,4,1,2007,1883.000000,1883.0,NaN,bbca52c92d682fd2accfbc25733650bb,"s5:c0803:v1883.0,",None,None,0
1,2,4,1,2008,2019.287306,NaN,NaN,341d23d2a89ce241357e4564aa64bf19,"a1:v32241.5,a23:v2019.29,",None,None,0
2,3,4,1,2009,2165.438781,62600.0,NaN,ce47b1ca9d513dcc48da9401e0dd266e,"s5:c0803:v62600.0,a23:v2165.44,",None,None,0
3,4,4,1,2010,2322.168372,17600.0,NaN,a9fc7d058c9a6d1d5937ee7033663c07,"s5:c0803:v17600.0,a23:v2322.17,",None,None,0
4,5,4,1,2011,2490.241697,46110.0,NaN,3cb16c589b696420a31bb344d49c53a1,"s5:c0803:v46110.0,a23:v2490.24,",None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4878,44764,990,2,2015,93592.118473,NaN,NaN,989df41d34d152dad2c6b91841c3f634,"a1:v174357.53,a23:v93592.12,",None,None,0
4879,44765,990,2,2016,44912.361244,NaN,NaN,664273dfc3e41c93ba29d4033e211362,"a1:v44912.36,",None,None,0
4880,44766,990,2,2017,59497.828669,2980.0,NaN,f30760f58f476ab81f0420ce04312af9,"s5:c0803:v2980.0,a23:v59497.83,",None,None,0
4881,44767,990,2,2018,74083.296094,NaN,NaN,d94790dbfc5fbf257756702a9715aa8a,NULL,None,None,0


In [111]:
df["factor"] = df["factor"].astype("category")

In [112]:
pivot = pd.pivot_table(df,
                       columns=["factor","year"],
                       values=["res"],
                       aggfunc=np.sum,
                       fill_value=np.nan)
pivot.rename(index={'res': 'world'}, inplace=True)
pivot

factor             1                                                          \
year            2007          2008          2009          2010          2011   
world   1.818306e+10  1.686853e+10  1.833181e+10  1.972401e+10  2.039291e+10   

factor                                                                        \
year            2012          2013          2014          2015          2016   
world   1.958085e+10  2.208439e+10  2.203008e+10  2.170082e+10  2.281889e+10   

factor  ...             2                                            \
year    ...          2010          2011          2012          2013   
world   ...  1.250882e+10  1.371111e+10  1.321901e+10  1.420963e+10   

factor                                                                        \
year            2014          2015          2016          2017          2018   
world   1.537200e+10  1.441794e+10  1.508187e+10  1.537707e+10  1.625388e+10   

factor                
year            2019  
world   1.651703e+10  

[1 rows x 26 columns]

Создадим переменные с факторами и годами, чтобы было проще к ним обращаться

In [113]:
factors = [1]*13+[2]*13+[6]*13
years = [year for year in range(2007, 2020)]

По заданию необходимо соединить предыдущую таблицу и фактор 6. Воспользуемся Join

In [114]:
factor_1 = pivot[1]
factor_2 = pivot[2]
df = factor_1.join(factor_2,rsuffix='right', lsuffix='left').join(factor_2/factor_1,rsuffix='right', lsuffix='left')
df.columns = pd.MultiIndex.from_tuples(zip(factors, years*3), 
                                       names=['factor', 'year'])
df

factor             1                                                          \
year            2007          2008          2009          2010          2011   
world   1.818306e+10  1.686853e+10  1.833181e+10  1.972401e+10  2.039291e+10   

factor                                                                        \
year            2012          2013          2014          2015          2016   
world   1.958085e+10  2.208439e+10  2.203008e+10  2.170082e+10  2.281889e+10   

factor  ...         6                                                    \
year    ...      2010      2011      2012      2013      2014      2015   
world   ...  0.634192  0.672347  0.675099  0.643424  0.697773  0.664396   

factor                                        
year        2016      2017    2018      2019  
world   0.660938  0.662354  0.6705  0.647046  

[1 rows x 39 columns]

Экспортируем полученный файл в Excel. Первая часть задания готова

In [115]:
df.to_excel("report.xlsx")

Вспомогательная функция для расчета совокупного среднегодового темпа роста.

[Видео-урок по CAGR на PYTHON](https://www.youtube.com/watch?v=e_cpNUt13UE)

Здесь зададим количество периодов по умолчанию равное двум (так как пока сравниваем 2 соседних периода попарно). В конце передадим в этот параметр весь период

In [116]:
def cagr(start, end, periods=2):
    return (end / start) ** (1 / (periods - 1)) - 1

Избавимся от 1 и 2 фактора, а также от называний. Приведем данные к массиву Numpy и к списку, чтобы удобнее было делать Dataframe. Также транспонируем его

In [117]:
df_6 = df[6].iloc[[0]]
df_6 = df_6.to_numpy().tolist()
new_df = pd.DataFrame(df_6, columns=[col for col in range(13)])
new_df = new_df.transpose()
new_df

,0
0,0.552081
1,0.688368
2,0.656792
3,0.634192
4,0.672347
5,0.675099
6,0.643424
7,0.697773
8,0.664396
9,0.660938


Создадим новый столбец, в который передадим вспомогательную функцию, которая считает среднегодовой темп роста. 

Обращаем внимание, что для первого периода отсутствует результат (что и логично, ведь перед ним нет периода для сравнения). Заменим их на NaN. Аналогично поступим с последним элементом

In [118]:
new_df['cagr'] = cagr(new_df-new_df.diff().fillna(new_df),new_df)
new_df.loc[0,'cagr'] =  new_df.loc[12,'cagr'] = np.nan
new_df

,0,cagr
0,0.552081,NaN
1,0.688368,0.246859
2,0.656792,-0.045870
3,0.634192,-0.034409
4,0.672347,0.060162
5,0.675099,0.004093
6,0.643424,-0.046919
7,0.697773,0.084468
8,0.664396,-0.047834
9,0.660938,-0.005204


In [119]:
new_df[0] = years
new_df.columns=['Year', 'World Result']
final_df=pd.DataFrame(new_df)
final_df.index=pd.MultiIndex.from_tuples(zip([6]*13),
                                         names=['Factor'])
final_df

,Year,World Result
Factor,,
6,2007,NaN
6,2008,0.246859
6,2009,-0.045870
6,2010,-0.034409
6,2011,0.060162
6,2012,0.004093
6,2013,-0.046919
6,2014,0.084468
6,2015,-0.047834


Для записи данных в Word понадобится знать размерность последнего Dataframe и наименование фактора

In [120]:
rows = final_df.shape[0]
cols = final_df.shape[1]
factor = final_df.index[0][0]

In [121]:
doc = docx.Document()
doc.add_heading('CAGR analyze', 0)
table = doc.add_table(rows+1, cols+1)
table.cell(0,0).text = "Factor"
table.cell(0,1).text = "Year"
table.cell(0,2).text = "World Result"

for i in range(rows):
  table.cell(i+1,0).text = str(factor)
  table.cell(i+1,1).text = str(years[i])
  table.cell(i+1,2).text = str(round(final_df.values[i,1], 2))

# в примере ячейка с фактором 6 объеденина и отформатирована по центру, сделаем так же:

merge_start = table.cell(1, 0)
merge_end = table.cell(rows, 0)
A = merge_start.merge(merge_end)
A.text = str(factor)
A.vertical_alignment = docx.enum.table.WD_ALIGN_VERTICAL.CENTER

condition = round(cagr(final_df.values[1,1],
                       final_df.values[rows-2,1],
                       len(years)-2), 2)

if condition>0:
    cagr="increased"
else:
    cagr="decreased"
    
doc.add_paragraph(f'Factor 6 {cagr} by avg {condition}% every year from {years[1]} to {years[11]}')
doc.save('report.docx')